## Провести сабмит на основе 2-х этапного подхода: модели для генерации кандидатов, реранжирующий бустинг

На первом этапе для генерации кандидатов используем  колаборативную фильтрацию -> далее делаем реранжируюший бустинг  


In [ ]:
import pandas as pd
import numpy as np

## Сначала накинем признаков для наших датасетов

In [ ]:
interactions = pd.read_csv('events.csv')
item_features = pd.read_csv('item_features.csv')
user_features = pd.read_csv('user_features.csv')
item_features.shape, user_features.shape, interactions.shape

((3706, 19), (6040, 3), (894149, 4))

In [ ]:
item_features.head()

,item_id,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17
0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
4,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
user_features.head()

,user_id,gender,age
0,4855,F,1
1,4065,M,56
2,3331,M,25
3,5373,M,45
4,2032,M,25


In [ ]:
interactions.head()

,user_id,item_id,rating,timestamp
0,0,1505,4,0
1,0,3669,3,1
2,0,584,4,2
3,0,3390,3,3
4,0,2885,4,4


In [ ]:
df = pd.merge(pd.merge(interactions, item_features, on = 'item_id', how = 'left'),
              user_features, on = 'user_id', how = 'left')

In [ ]:
df.head(15)

,user_id,item_id,rating,timestamp,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,...,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,gender,age
0,0,1505,4,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,M,35
1,0,3669,3,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,M,35
2,0,584,4,2,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,M,35
3,0,3390,3,3,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,M,35
4,0,2885,4,4,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,M,35
5,0,79,5,5,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,M,35
6,0,717,4,7,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,M,35
7,0,187,4,8,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,M,35
8,0,93,5,9,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,M,35
9,0,3016,5,10,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,M,35


## Для **Items**

% соотношение просмотров по полу

In [ ]:
genders = df.groupby('item_id')['gender'].value_counts(normalize=True).unstack().fillna(0)
genders

gender,F,M
item_id,,
0,0.283582,0.716418
1,0.285714,0.714286
2,0.307339,0.692661
3,0.486667,0.513333
4,0.204545,0.795455
...,...,...
3701,0.250478,0.749522
3702,0.156250,0.843750
3703,0.000000,1.000000


% соотношение по бакету возраста

In [ ]:
df['age_group'] = pd.cut(df['age'], bins=[0, 18, 25, 40, 100], labels=['0-18', '18-25', '25-40', '40+'])
ages = df.groupby('item_id')['age_group'].value_counts(normalize=True).unstack().fillna(0)
ages

age_group,0-18,18-25,25-40,40+
item_id,,,,
0,0.492537,0.246269,0.111940,0.149254
1,0.000000,0.357143,0.285714,0.357143
2,0.114679,0.380734,0.224771,0.279817
3,0.100000,0.193333,0.293333,0.413333
4,0.113636,0.363636,0.250000,0.272727
...,...,...,...,...
3701,0.336520,0.384321,0.172084,0.107075
3702,0.093750,0.062500,0.250000,0.593750
3703,0.000000,0.000000,0.000000,1.000000


In [ ]:
genre_columns = [col for col in item_features.columns if col.startswith('genre')]
genre_gender = df.groupby(genre_columns)['gender'].value_counts(normalize=True).unstack().fillna(0)
genre_gender

gender                                                                                                                                                          F  \
genre_0 genre_1 genre_2 genre_3 genre_4 genre_5 genre_6 genre_7 genre_8 genre_9 genre_10 genre_11 genre_12 genre_13 genre_14 genre_15 genre_16 genre_17             
0       0       0       0       0       0       0       0       0       0       0        0        0        0        0        0        0        1         0.119197   
                                                                                                                                      1        0         0.096591   
                                                                                                                             1        0        0         0.240573   
                                                                                                                    1        0        0        0         0.156406   
                                                                                                                                      1        0         0.166121   
...                                                                                                                                                           ...   
1       1       0       1       0       0       0       0       1       0       0        0        0        0        0        0        0        0         0.170732   
                                1       0       0       0       0       0       0        0        0        0        0        0        0        0         0.236052   
                1       0       0       0       0       0       0       0       0        0        0        0        0        0        0        0         0.182410   
                                                                                1        0        0        0        1        0        0        0         0.111712   
                        1       0       0       0       0       1       0       0        0        0        0        0        0        0        0         0.300813   

gender                                                                                                                                                          M  
genre_0 genre_1 genre_2 genre_3 genre_4 genre_5 genre_6 genre_7 genre_8 genre_9 genre_10 genre_11 genre_12 genre_13 genre_14 genre_15 genre_16 genre_17            
0       0       0       0       0       0       0       0       0       0       0        0        0        0        0        0        0        1         0.880803  
                                                                                                                                      1        0         0.903409  
                                                                                                                             1        0        0         0.759427  
                                                                                                                    1        0        0        0         0.843594  
                                                                                                                                      1        0         0.833879  
...                                                                                                                                                           ...  
1       1       0       1       0       0       0       0       1       0       0        0        0        0        0        0        0        0         0.829268  
                                1       0       0       0       0       0       0        0        0        0        0        0        0        0         0.763948  
                1       0       0       0       0       0       0       0       0        0        0        0        0        0        0        0         0.817590  
                                                  

In [ ]:
genre_age = df.groupby(genre_columns)['age_group'].value_counts(normalize=True).unstack().fillna(0)
genre_age

age_group                                                                                                                                                    0-18  \
genre_0 genre_1 genre_2 genre_3 genre_4 genre_5 genre_6 genre_7 genre_8 genre_9 genre_10 genre_11 genre_12 genre_13 genre_14 genre_15 genre_16 genre_17             
0       0       0       0       0       0       0       0       0       0       0        0        0        0        0        0        0        0         0.000000   
                                                                                                                                               1         0.116640   
                                                                                                                                      1        0         0.098864   
                                                                                                                                               1         0.000000   
                                                                                                                             1        0        0         0.184075   
...                                                                                                                                                           ...   
1       1       1       1       1       1       1       1       1       1       1        1        1        1        1        0        1        1         0.000000   
                                                                                                                             1        0        0         0.000000   
                                                                                                                                               1         0.000000   
                                                                                                                                      1        0         0.000000   
                                                                                                                                               1         0.000000   

age_group                                                                                                                                                   18-25  \
genre_0 genre_1 genre_2 genre_3 genre_4 genre_5 genre_6 genre_7 genre_8 genre_9 genre_10 genre_11 genre_12 genre_13 genre_14 genre_15 genre_16 genre_17             
0       0       0       0       0       0       0       0       0       0       0        0        0        0        0        0        0        0         0.000000   
                                                                                                                                               1         0.329662   
                                                                                                                                      1        0         0.197727   
                                                                                                                                               1         0.000000   
                                                                                                                             1        0        0         0.417798   
...                                                                                                                                                           ...   
1       1       1       1       1       1       1       1       1       1       1        1        1        1        1        0        1        1         0.000000   
                                                                                                                             1        0        0         0.000000   
                                                                                                                                               1         0.000000   
                                       

Показатели рейтинга фильмов

In [ ]:
item_stats = interactions.groupby('item_id').agg(
    avg_item_rating=('rating', 'mean'),
    item_rating_median=('rating', 'median'),
    item_rating_count=('rating', 'count'),
    item_rating_quantile_25=('rating', lambda x: x.quantile(0.25)),
    item_rating_quantile_75=('rating', lambda x: x.quantile(0.75))
).reset_index()
item_stats

,item_id,avg_item_rating,item_rating_median,item_rating_count,item_rating_quantile_25,item_rating_quantile_75
0,0,2.395522,2.0,134,1.0,3.0
1,1,3.285714,3.0,14,3.0,4.0
2,2,3.655963,4.0,218,3.0,4.0
3,3,3.966667,4.0,150,3.0,5.0
4,4,3.590909,4.0,44,3.0,4.0
...,...,...,...,...,...,...
3685,3701,3.564054,4.0,523,3.0,4.0
3686,3702,2.656250,3.0,32,2.0,3.0
3687,3703,3.000000,3.0,1,3.0,3.0
3688,3704,3.753425,4.0,73,3.0,4.0


In [ ]:
df = pd.merge(df, item_stats, on='item_id', how='left')

In [ ]:
ages.columns = ['under_18_rate', '18-25_rate', '25-40_rate', '40plus_rate']

In [ ]:
genders.columns = [ 'F_rate', 'M_rate']

In [ ]:
item_features = pd.merge(item_features, item_stats, on='item_id', how='left')

In [ ]:
item_features = pd.merge(item_features, genders, on='item_id', how='left')
item_features = pd.merge(item_features, ages, on='item_id', how='left')

Все фичи для Item'ов

In [ ]:
item_features

,item_id,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,...,item_rating_median,item_rating_count,item_rating_quantile_25,item_rating_quantile_75,F_rate,M_rate,under_18_rate,18-25_rate,25-40_rate,40plus_rate
0,0,0,1,0,1,1,0,0,0,1,...,2.0,134.0,1.0,3.0,0.283582,0.716418,0.492537,0.246269,0.111940,0.149254
1,1,0,0,0,0,0,0,0,1,0,...,3.0,14.0,3.0,4.0,0.285714,0.714286,0.000000,0.357143,0.285714,0.357143
2,2,0,0,0,0,0,0,0,1,0,...,4.0,218.0,3.0,4.0,0.307339,0.692661,0.114679,0.380734,0.224771,0.279817
3,3,0,0,0,0,0,0,0,1,0,...,4.0,150.0,3.0,5.0,0.486667,0.513333,0.100000,0.193333,0.293333,0.413333
4,4,0,0,0,0,0,0,0,1,0,...,4.0,44.0,3.0,4.0,0.204545,0.795455,0.113636,0.363636,0.250000,0.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701,3701,0,0,1,0,0,0,0,0,0,...,4.0,523.0,3.0,4.0,0.250478,0.749522,0.336520,0.384321,0.172084,0.107075
3702,3702,0,0,0,0,0,0,0,0,0,...,3.0,32.0,2.0,3.0,0.156250,0.843750,0.093750,0.062500,0.250000,0.593750
3703,3703,0,0,0,0,0,0,0,1,0,...,3.0,1.0,3.0,3.0,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
3704,3704,0,0,0,0,0,0,0,1,0,...,4.0,73.0,3.0,4.0,0.383562,0.616438,0.082192,0.383562,0.260274,0.273973


## Фичи для users по выставленным рейтингам

In [ ]:
user_ratings = pd.concat(
    [
        df.groupby('user_id').rating.count(),
        df.groupby('user_id').rating.mean().round(4),
        df.groupby('user_id').rating.std().round(4),
        df.groupby('user_id').rating.quantile([0, 0.25, 0.5, 0.75, 1]).unstack(level=1)
    ],
    axis=1
)

user_ratings.columns = [
    'user_rating_count',
    'user_rating_mean',
    'user_rating_std',
    'user_rating_min',
    'user_rating_q25',
    'user_rating_q50',
    'user_rating_q75',
    'user_rating_max'
]

user_ratings.head()

,user_rating_count,user_rating_mean,user_rating_std,user_rating_min,user_rating_q25,user_rating_q50,user_rating_q75,user_rating_max
user_id,,,,,,,,
0,287,3.9791,0.8965,1.0,3.0,4.0,5.0,5.0
1,261,3.6475,0.9719,1.0,3.0,4.0,4.0,5.0
2,143,3.7972,0.7077,1.0,3.0,4.0,4.0,5.0
3,231,3.3506,1.0480,1.0,3.0,3.0,4.0,5.0
4,107,4.0467,0.8398,2.0,3.0,4.0,5.0,5.0


## Доли просмотренных жанров

In [ ]:
user_genres = df.groupby('user_id')[genre_columns].mean().round(4)

user_genres.columns = [f"user_genre{i}_share" for i in np.arange(len(genre_columns))]

user_genres.head()

,user_genre0_share,user_genre1_share,user_genre2_share,user_genre3_share,user_genre4_share,user_genre5_share,user_genre6_share,user_genre7_share,user_genre8_share,user_genre9_share,user_genre10_share,user_genre11_share,user_genre12_share,user_genre13_share,user_genre14_share,user_genre15_share,user_genre16_share,user_genre17_share
user_id,,,,,,,,,,,,,,,,,,
0,0.3310,0.2125,0.0906,0.1498,0.3659,0.0697,0.0000,0.2474,0.0662,0.0139,0.0767,0.0627,0.0453,0.1882,0.1638,0.2265,0.0592,0.0139
1,0.3257,0.1303,0.0268,0.0345,0.2605,0.1226,0.0115,0.2644,0.0268,0.0192,0.1226,0.0192,0.0345,0.0920,0.2337,0.4023,0.0536,0.0192
2,0.5804,0.2867,0.0559,0.0559,0.2448,0.0769,0.0070,0.2098,0.0280,0.0070,0.0769,0.0210,0.0280,0.1049,0.2937,0.3077,0.0629,0.0140
3,0.5887,0.2684,0.0390,0.0693,0.3636,0.0736,0.0000,0.1299,0.0390,0.0000,0.0779,0.0173,0.0260,0.0693,0.3290,0.2857,0.0563,0.0173
4,0.4019,0.1121,0.0374,0.0374,0.3645,0.0935,0.0000,0.3364,0.0093,0.0000,0.0187,0.0280,0.0187,0.1215,0.1776,0.2617,0.0561,0.0280


In [ ]:
user_features = pd.concat(
    [user_ratings, user_genres],
    axis=1
)

In [ ]:
user_features

,user_rating_count,user_rating_mean,user_rating_std,user_rating_min,user_rating_q25,user_rating_q50,user_rating_q75,user_rating_max,user_genre0_share,user_genre1_share,...,user_genre8_share,user_genre9_share,user_genre10_share,user_genre11_share,user_genre12_share,user_genre13_share,user_genre14_share,user_genre15_share,user_genre16_share,user_genre17_share
user_id,,,,,,,,,,,,,,,,,,,,,
0,287,3.9791,0.8965,1.0,3.0,4.0,5.0,5.0,0.3310,0.2125,...,0.0662,0.0139,0.0767,0.0627,0.0453,0.1882,0.1638,0.2265,0.0592,0.0139
1,261,3.6475,0.9719,1.0,3.0,4.0,4.0,5.0,0.3257,0.1303,...,0.0268,0.0192,0.1226,0.0192,0.0345,0.0920,0.2337,0.4023,0.0536,0.0192
2,143,3.7972,0.7077,1.0,3.0,4.0,4.0,5.0,0.5804,0.2867,...,0.0280,0.0070,0.0769,0.0210,0.0280,0.1049,0.2937,0.3077,0.0629,0.0140
3,231,3.3506,1.0480,1.0,3.0,3.0,4.0,5.0,0.5887,0.2684,...,0.0390,0.0000,0.0779,0.0173,0.0260,0.0693,0.3290,0.2857,0.0563,0.0173
4,107,4.0467,0.8398,2.0,3.0,4.0,5.0,5.0,0.4019,0.1121,...,0.0093,0.0000,0.0187,0.0280,0.0187,0.1215,0.1776,0.2617,0.0561,0.0280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,90,4.0333,0.9881,1.0,3.0,4.0,5.0,5.0,0.4444,0.2111,...,0.0444,0.0222,0.0667,0.0111,0.0000,0.0556,0.5556,0.2111,0.1222,0.0000
6036,29,4.0000,0.9636,2.0,4.0,4.0,5.0,5.0,0.1724,0.1034,...,0.0690,0.0345,0.0345,0.0345,0.0345,0.1034,0.1379,0.1379,0.1379,0.0345
6037,90,3.6778,0.9220,1.0,3.0,4.0,4.0,5.0,0.1556,0.0889,...,0.0000,0.0333,0.0556,0.0444,0.0444,0.1111,0.0889,0.1333,0.0556,0.0111


In [ ]:
item_features.columns

Index(['item_id', 'genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4',
       'genre_5', 'genre_6', 'genre_7', 'genre_8', 'genre_9', 'genre_10',
       'genre_11', 'genre_12', 'genre_13', 'genre_14', 'genre_15', 'genre_16',
       'genre_17', 'avg_item_rating', 'item_rating_median',
       'item_rating_count', 'item_rating_quantile_25',
       'item_rating_quantile_75', 'F_rate', 'M_rate', 'under_18_rate',
       '18-25_rate', '25-40_rate', '40plus_rate'],
      dtype='object')

In [ ]:
final_df = pd.merge(df, user_features, on = 'user_id', how = 'left')
final_df =  pd.merge(final_df, item_features[['item_id', 'avg_item_rating', 'item_rating_median',
       'item_rating_count', 'item_rating_quantile_25',
       'item_rating_quantile_75','F_rate', 'M_rate', 'under_18_rate', '18-25_rate',
       '25-40_rate', '40plus_rate']], on = 'item_id', how = 'left')

## Финальная таблица со всеми фичами

In [ ]:
final_df.head(10)

,user_id,item_id,rating,timestamp,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,...,item_rating_median_y,item_rating_count_y,item_rating_quantile_25_y,item_rating_quantile_75_y,F_rate,M_rate,under_18_rate,18-25_rate,25-40_rate,40plus_rate
0,0,1505,4,0,1,0,0,0,0,1,...,5.0,1505.0,4.0,5.0,0.205316,0.794684,0.165449,0.386047,0.213953,0.234551
1,0,3669,3,1,0,0,0,0,1,0,...,3.0,737.0,3.0,4.0,0.388060,0.611940,0.241520,0.370421,0.179104,0.208955
2,0,584,4,2,1,1,0,0,0,0,...,4.0,2423.0,3.0,4.0,0.213785,0.786215,0.235246,0.378869,0.193974,0.191911
3,0,3390,3,3,0,0,0,0,1,1,...,4.0,1019.0,3.0,4.0,0.271835,0.728165,0.244357,0.474975,0.162905,0.117763
4,0,2885,4,4,1,0,0,0,0,1,...,3.0,695.0,3.0,4.0,0.189928,0.810072,0.161151,0.427338,0.181295,0.230216
5,0,79,5,5,0,0,0,0,1,0,...,4.0,741.0,3.0,4.0,0.269906,0.730094,0.273954,0.388664,0.168691,0.168691
6,0,717,4,7,0,0,0,0,0,0,...,4.0,735.0,3.0,4.0,0.296599,0.703401,0.091156,0.446259,0.247619,0.214966
7,0,187,4,8,0,0,0,0,0,0,...,4.0,314.0,4.0,4.0,0.257962,0.742038,0.101911,0.356688,0.216561,0.324841
8,0,93,5,9,0,0,0,0,0,0,...,4.0,818.0,4.0,5.0,0.270171,0.729829,0.161369,0.317848,0.244499,0.276284
9,0,3016,5,10,0,0,0,0,0,0,...,4.0,318.0,3.0,5.0,0.201258,0.798742,0.053459,0.496855,0.273585,0.176101


## Сплитуем на train и test

In [ ]:

train_size = 0.7
train, test = train_test_split(final_df, train_size=train_size, shuffle=False)
print(f'Train size: {len(train)}')
print(f'Test size: {len(test)}')

Train size: 625904
Test size: 268245


In [ ]:
def hybrid_filtration(interaction_matrix_scaled, user_similarity_df, item_similarity_df,  client, num_recommendations):
    interacted_items = interaction_matrix_scaled.loc[client][interaction_matrix_scaled.loc[client] > 0].index.tolist()
    # Функция для получения рекомендаций на основе User-based CF
    def get_user_based_recommendations(user_id, num_recommendations=num_recommendations):
        similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:num_recommendations + 1]
        recommended_items = interaction_matrix_scaled.loc[similar_users].sum().sort_values(ascending=False).index[:num_recommendations]
        return recommended_items

    # Функция для получения рекомендаций на основе Item-based CF
    def get_item_based_recommendations(item_id, num_recommendations=num_recommendations*3):
        similar_items = item_similarity_df[item_id].sort_values(ascending=False).index[1:num_recommendations + 1]
        return similar_items

    def get_all_item_recomendations(user_id, num_recommendations=num_recommendations):
        item_based_recommendations = []
        for item_id in interacted_items:
            item_based_recommendations.extend(get_item_based_recommendations(item_id, num_recommendations * 3))
        filtered_based_recommendations = [item for item in item_based_recommendations if item not in interacted_items]
        recommendation_counts = Counter(filtered_based_recommendations)
        top_recommendations = recommendation_counts.most_common(num_recommendations)
        return [item for item, count in top_recommendations]

    def hybrid_recommendation(user_id, num_recommendations=num_recommendations):
        user_based_recommendations = get_user_based_recommendations(user_id, num_recommendations)
        user_filtered_based_recommendations = [item for item in user_based_recommendations if item not in interacted_items]
        item_based_recommendations = get_all_item_recomendations(user_id, num_recommendations)
        combined_recommendations = set(user_based_recommendations).union(set(item_based_recommendations))
        return tuple([user_filtered_based_recommendations, item_based_recommendations])

    return  hybrid_recommendation(client)

In [ ]:
from tqdm import tqdm
def get_filter_recs(data):

    # Создание матрицы взаимодействий
    interaction_matrix = data.pivot(index='user_id', columns='item_id', values='timestamp').fillna(0)

    # Коллаборативная фильтрация (User-based)
    user_similarity = cosine_similarity(interaction_matrix)
    user_similarity_df = pd.DataFrame(user_similarity, index=interaction_matrix.index, columns=interaction_matrix.index)

    # Коллаборативная фильтрация (Item-based)
    item_similarity = cosine_similarity(interaction_matrix.T)
    item_similarity_df = pd.DataFrame(item_similarity, index=interaction_matrix.columns, columns=interaction_matrix.columns)
    recommendation_dict ={}
    for user in tqdm(data.user_id.unique()):
        recommendation_dict[user] = hybrid_filtration(interaction_matrix, user_similarity_df, item_similarity_df,  user, num_recommendations = 50 )
    return recommendation_dict

In [ ]:
recommendation_dict = get_filter_recs(train)

100%|██████████| 4191/4191 [11:13<00:00,  6.22it/s]


In [ ]:
unique_users = final_df['user_id'].unique()
unique_items = final_df['item_id'].unique()

all_pairs = pd.MultiIndex.from_product([unique_users, unique_items], names=['user_id', 'item_id']).to_frame(index=False)

train_pairs = train[['user_id', 'item_id']]

val = all_pairs.merge(train_pairs, on=['user_id', 'item_id'], how='left', indicator=True)
val = val[val['_merge'] == 'left_only'].drop(columns=['_merge'])

In [ ]:
def add_recommendation_columns(data, recommendation_dict):
    recomm_user = []
    recomm_item = []
    for index, row in tqdm(data.iterrows()):
        user = row['user_id']
        item = row['item_id']
        if user in recommendation_dict.keys():
            if item in recommendation_dict[user][0]:
                recomm_user.append(1)
            else:
                recomm_user.append(0)
            if item in recommendation_dict[user][1]:
                recomm_item.append(1)
            else:
                recomm_item.append(0)
        else:
            recomm_user.append(0)
            recomm_item.append(0)

    return recomm_user, recomm_item

In [ ]:
val['recommendation_user'], val['recommendation_item'] = add_recommendation_columns(val, recommendation_dict)

21661696it [13:39, 26416.77it/s]


In [ ]:
rating_dict = test.set_index(['user_id', 'item_id'])['rating'].to_dict()

# Создаем y
def get_y(row):
    user_id = row['user_id']
    item_id = row['item_id']

    # Проверяем, есть ли пара
    if (user_id, item_id) in rating_dict:
        rating = rating_dict[(user_id, item_id)]
        if rating <= 2:
            return 0
        elif rating == 3:
            return 0.5
        elif rating >= 4:
            return 1
    return 0.2

val['y'] = val.apply(get_y, axis=1)

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
val.head()

,user_id,item_id,recommendation_user,recommendation_item,y
287,0,738,0,0,0.25
288,0,488,0,0,0.25
289,0,702,0,0,0.25
290,0,378,0,0,0.25
291,0,174,0,0,0.25


In [ ]:
final_df.shape

(894149, 67)

In [ ]:
val_sampled = val.sample(500000)

In [ ]:
final_df.columns

Index(['user_id', 'item_id', 'rating', 'timestamp', 'genre_0', 'genre_1',
       'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7',
       'genre_8', 'genre_9', 'genre_10', 'genre_11', 'genre_12', 'genre_13',
       'genre_14', 'genre_15', 'genre_16', 'genre_17', 'gender', 'age',
       'age_group', 'avg_item_rating_x', 'item_rating_median_x',
       'item_rating_count_x', 'item_rating_quantile_25_x',
       'item_rating_quantile_75_x', 'user_rating_count', 'user_rating_mean',
       'user_rating_std', 'user_rating_min', 'user_rating_q25',
       'user_rating_q50', 'user_rating_q75', 'user_rating_max',
       'user_genre0_share', 'user_genre1_share', 'user_genre2_share',
       'user_genre3_share', 'user_genre4_share', 'user_genre5_share',
       'user_genre6_share', 'user_genre7_share', 'user_genre8_share',
       'user_genre9_share', 'user_genre10_share', 'user_genre11_share',
       'user_genre12_share', 'user_genre13_share', 'user_genre14_share',
       'user_genre15

In [ ]:
items = final_df[['item_id','genre_0', 'genre_1',
       'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7',
       'genre_8', 'genre_9', 'genre_10', 'genre_11', 'genre_12', 'genre_13',
       'genre_14', 'genre_15', 'genre_16', 'genre_17', 'gender', 'age',
       'age_group', 'avg_item_rating_x', 'item_rating_median_x',
       'item_rating_count_x', 'item_rating_quantile_25_x',
       'item_rating_quantile_75_x','F_rate',
       'M_rate', 'under_18_rate', '18-25_rate', '25-40_rate', '40plus_rate']].drop_duplicates()

In [ ]:
items.shape

(39321, 33)

In [ ]:
users = final_df[['user_id', 'gender', 'age','user_rating_count', 'user_rating_mean',
       'user_rating_std', 'user_rating_min', 'user_rating_q25',
       'user_rating_q50', 'user_rating_q75', 'user_rating_max',
       'user_genre0_share', 'user_genre1_share', 'user_genre2_share',
       'user_genre3_share', 'user_genre4_share', 'user_genre5_share',
       'user_genre6_share', 'user_genre7_share', 'user_genre8_share',
       'user_genre9_share', 'user_genre10_share', 'user_genre11_share',
       'user_genre12_share', 'user_genre13_share', 'user_genre14_share',
       'user_genre15_share', 'user_genre16_share', 'user_genre17_share']].drop_duplicates()

In [ ]:
val_sampled = pd.merge(val_sampled, users, on = 'user_id', how = 'left')

In [ ]:
val_sampled = pd.merge(val_sampled, items, on = 'item_id', how = 'left')

In [ ]:
X = val_sampled.drop(columns=['y'])
y = val_sampled['y']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)